<H1>Problem Statement</H1>
Your client is a large MNC and they have 9 broad verticals across the organisation. One of the problem your client is facing is around identifying the right people for promotion (only for manager position and below) and prepare them in time. Currently the process, they are following is:

They first identify a set of employees based on recommendations/ past performance
Selected employees go through the separate training and evaluation program for each vertical. These programs are based on the required skill of each vertical
At the end of the program, based on various factors such as training performance, KPI completion (only employees with KPIs completed greater than 60% are considered) etc., employee gets promotion
For above mentioned process, the final promotions are only announced after the evaluation and this leads to delay in transition to their new roles. Hence, company needs your help in identifying the eligible candidates at a particular checkpoint so that they can expedite the entire promotion cycle.

They have provided multiple attributes around Employee's past and current performance along with demographics. Now, The task is to predict whether a potential promotee at checkpoint in the test set will be promoted or not after the evaluation process.

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!cp  /content/drive/Hackthon/WNS/* .
!ls -l

In [0]:
!ls -l


total 6164
-rw-r--r-- 1 root root    2495 Sep 14 13:24 adc.json
drwxr-xr-x 4 root root    4096 Sep 14 14:12 catboost_info
drwxr-xr-x 2 root root    4096 Sep 14 13:24 drive
drwxr-xr-x 2 root root    4096 Sep 12 16:53 sample_data
-rw-r--r-- 1 root root 1565147 Sep 14 13:25 test_2umaH9m.csv
-rw-r--r-- 1 root root 3759647 Sep 14 13:25 train_LZdllcl.csv
-rw-r--r-- 1 root root  231556 Sep 14 18:18 WNS_catBoost_01.csv
-rw-r--r-- 1 root root  231556 Sep 14 19:48 WNS_catBoost_Test_01.csv
-rw-r--r-- 1 root root  231556 Sep 14 20:03 WNS_catBoost_Test_03.csv
-rw-r--r-- 1 root root  231556 Sep 14 20:53 WNS_catBoost_Test_03_stacked.csv
-rw-r--r-- 1 root root   29765 Sep 14 13:25 WNS_Traine_01.ipynb


In [0]:
!pip install catboost
!pip install ipywidgets
!pip install lightgbm
!pip install xgboost     


<h1>3. Exploratory Data Analysis</h1>
<h3>3.1. Import All required Classed first</h3>


In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,roc_auc_score,mean_squared_error,f1_score
from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.cross_validation import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from numpy import argmax
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool, cv
from lightgbm import LGBMClassifier

/usr/local/lib/python3.6/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [0]:
SEED = 0

In [0]:
Data = pd.read_csv("train_LZdllcl.csv")

In [4]:
print(Data.shape)
print(Data.columns)

(54808, 14)
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')


In [5]:
print(Data.describe())

        employee_id  no_of_trainings           age  previous_year_rating  \
count  54808.000000     54808.000000  54808.000000          50684.000000   
mean   39195.830627         1.253011     34.803915              3.329256   
std    22586.581449         0.609264      7.660169              1.259993   
min        1.000000         1.000000     20.000000              1.000000   
25%    19669.750000         1.000000     29.000000              3.000000   
50%    39225.500000         1.000000     33.000000              3.000000   
75%    58730.500000         1.000000     39.000000              4.000000   
max    78298.000000        10.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  \
count       54808.000000   54808.000000  54808.000000        54808.000000   
mean            5.865512       0.351974      0.023172           63.386750   
std             4.265094       0.477590      0.150450           13.371559   
min    

In [6]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               52399
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    50684
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [0]:
values={"previous_year_rating":Data.previous_year_rating.mean(),"education":"O"}
Data.fillna(value=values,inplace=True)

In [8]:
Data.isna().values.any()


False

In [9]:
print(len(Data))
print(len(Data.loc[Data["is_promoted"]==0]))
print(len(Data.loc[Data["is_promoted"]==1]))
print(len(Data))

54808
50140
4668
54808


In [10]:
50140/54808*100

91.48299518318494

In [11]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               54808
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    54808
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [12]:
Data.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [0]:
#data_xs = Data.drop(['employee_id',"is_promoted","education"],axis=1)
#data_xs = Data.drop(['employee_id',"is_promoted"],axis=1)
#data_xs = Data.drop(['employee_id',"is_promoted","gender","recruitment_channel","region"],axis=1)
#data_xs = Data.loc[:,["department","avg_training_score","length_of_service"]]
data_xs = Data.loc[:,["department","avg_training_score","length_of_service"]]

In [14]:
data_xs.head()


,department,avg_training_score,length_of_service
0,Sales & Marketing,49,8
1,Operations,60,4
2,Sales & Marketing,50,7
3,Sales & Marketing,50,10
4,Technology,73,2


In [0]:
categorical_features_indices = np.array([0,1,2,3,4,6,8,10,11])
#categorical_features_indices = np.array([0,1,3,6,7])
categorical_features_indices = np.array([0])

In [0]:
def model_evalution(model):
  print("####################### model Evalution started #######################")
  train_pre = model.predict(x_train)
  test_pre = model.predict(x_test)
  train_pro = model.predict_proba(x_train)
  test_pro = model.predict_proba(x_test)
  
  print("Train Accuracy: {0} \t Test Accuracy: {1}".format(accuracy_score(y_train, train_pre),accuracy_score(y_test,test_pre)))
  print("Train Loss: {0} \t Test Loss: {1}".format(mean_squared_error(y_train, train_pre),mean_squared_error(y_test,test_pre)))
  print("Train AUC: {0} \t Test AUC: {1}".format(roc_auc_score(y_train, train_pro[:,1]),roc_auc_score(y_test,test_pro[:,1])))
  print("Train f1: {0} \t Test F1: {1}".format(f1_score(y_train, train_pre),f1_score(y_test,test_pre)))
  print("Train Confusion Matrix: \n{0} \n Test Confusion Matrix: \n{1}".format(confusion_matrix(y_train, train_pre),confusion_matrix(y_test,test_pre)))
  #f1_score
  
  

In [0]:
x_train,x_test,y_train,y_test = train_test_split(data_xs,Data["is_promoted"], test_size = 0.3,random_state=SEED)

In [0]:
del(catboost_model)

In [56]:
#catboost_model = CatBoostClassifier(class_weights=[1,9])
catboost_model = CatBoostClassifier(iterations=200,verbose=True,learning_rate=0.01,class_weights=[1,2],depth=3,l2_leaf_reg=100,bagging_temperature=0.2)
#catboost_model = CatBoostClassifier(iterations=50,verbose=True,eval_metric="F1",learning_rate=0.5,class_weights=[1,3],depth=3,l2_leaf_reg=50,bagging_temperature=0.5)--->0.5139
catboost_model.fit(x_train, y_train,cat_features=categorical_features_indices,eval_set=(x_test, y_test),use_best_model=False)
#catboost_model.fit(x_train, y_train,cat_features=categorical_features_indices)
model_evalution(catboost_model)

0:	learn: 0.6862780	test: 0.6862458	best: 0.6862458 (0)	total: 65.7ms	remaining: 13.1s
1:	learn: 0.6796167	test: 0.6795534	best: 0.6795534 (1)	total: 155ms	remaining: 15.3s
2:	learn: 0.6731162	test: 0.6730202	best: 0.6730202 (2)	total: 243ms	remaining: 16s
3:	learn: 0.6667713	test: 0.6666419	best: 0.6666419 (3)	total: 316ms	remaining: 15.5s
4:	learn: 0.6606017	test: 0.6604457	best: 0.6604457 (4)	total: 413ms	remaining: 16.1s
5:	learn: 0.6548692	test: 0.6546716	best: 0.6546716 (5)	total: 501ms	remaining: 16.2s
6:	learn: 0.6488991	test: 0.6486648	best: 0.6486648 (6)	total: 578ms	remaining: 15.9s
7:	learn: 0.6429887	test: 0.6427361	best: 0.6427361 (7)	total: 666ms	remaining: 16s
8:	learn: 0.6372524	test: 0.6369799	best: 0.6369799 (8)	total: 755ms	remaining: 16s
9:	learn: 0.6318419	test: 0.6315499	best: 0.6315499 (9)	total: 830ms	remaining: 15.8s
10:	learn: 0.6264505	test: 0.6261277	best: 0.6261277 (10)	total: 899ms	remaining: 15.4s
11:	learn: 0.6214321	test: 0.6210839	best: 0.6210839 (11)

In [20]:
catboost_model.feature_importances_

array([39.1866413 , 60.56442187,  0.24893684])

In [0]:
train_pre = catboost_model.predict(x_train)


In [0]:
Data_test = pd.read_csv("test_2umaH9m.csv")

In [0]:
print(Data_test.shape)
print(Data_test.columns)

(23490, 13)
Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score'],
      dtype='object')


In [0]:
print(Data_test.describe())

        employee_id  no_of_trainings           age  previous_year_rating  \
count  23490.000000     23490.000000  23490.000000          21678.000000   
mean   39041.399149         1.254236     34.782929              3.339146   
std    22640.809201         0.600910      7.679492              1.263294   
min        3.000000         1.000000     20.000000              1.000000   
25%    19370.250000         1.000000     29.000000              3.000000   
50%    38963.500000         1.000000     33.000000              3.000000   
75%    58690.000000         1.000000     39.000000              4.000000   
max    78295.000000         9.000000     60.000000              5.000000   

       length_of_service  KPIs_met >80%   awards_won?  avg_training_score  
count       23490.000000   23490.000000  23490.000000        23490.000000  
mean            5.810387       0.358834      0.022776           63.263133  
std             4.207917       0.479668      0.149191           13.411750  
min        

In [0]:
Data.count()

employee_id             54808
department              54808
region                  54808
education               54808
gender                  54808
recruitment_channel     54808
no_of_trainings         54808
age                     54808
previous_year_rating    54808
length_of_service       54808
KPIs_met >80%           54808
awards_won?             54808
avg_training_score      54808
is_promoted             54808
dtype: int64

In [0]:
values={"previous_year_rating":Data_test.previous_year_rating.mean(),"education":"O"}
Data_test.fillna(value=values,inplace=True)

In [0]:
Data_test.isna().values.any()


False

In [0]:
Data_test.count()

employee_id             23490
department              23490
region                  23490
education               23490
gender                  23490
recruitment_channel     23490
no_of_trainings         23490
age                     23490
previous_year_rating    23490
length_of_service       23490
KPIs_met >80%           23490
awards_won?             23490
avg_training_score      23490
dtype: int64

In [0]:
Data_test.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,8724,Technology,region_26,Bachelor's,m,sourcing,1,24,3.339146,1,1,0,77
1,74430,HR,region_4,Bachelor's,f,other,1,31,3.000000,5,0,0,51
2,72255,Sales & Marketing,region_13,Bachelor's,m,other,1,31,1.000000,4,0,0,47
3,38562,Procurement,region_2,Bachelor's,f,other,3,31,2.000000,9,0,0,65
4,64486,Finance,region_29,Bachelor's,m,sourcing,1,30,4.000000,7,0,0,61


In [0]:
data_test_xs = Data_test.drop(['employee_id'],axis=1)
#data_test_xs = Data_test.drop(['employee_id',"education","gender","recruitment_channel","region"],axis=1)

In [0]:
y_test_pre = catboost_model.predict(data_test_xs)

In [0]:
Data_test["is_promoted"] = y_test_pre

In [0]:
Data_test.to_csv("WNS_catBoost_Test_05_30.csv",columns=["employee_id","is_promoted"],index=False)

In [0]:
from google.colab import files
files.download("WNS_catBoost_Test_05_30.csv")